In [1]:
%matplotlib inline

In [2]:
import numpy as np
from Pyfhel import Pyfhel

In [3]:
n = 2**13               # 2^13 = 8192

HE = Pyfhel()           # Creating empty Pyfhel object
bfv_params = {
    'scheme': 'BFV',    # can also be 'bfv'
    'n': n,             # Polynomial modulus degree, the num. of slots per plaintext,
                        #  of elements to be encoded in a single ciphertext in a
                        #  2 by n/2 rectangular matrix (mind this shape for rotations!)
                        #  Typ. 2^D for D in [10, 16]
    't': 65537,         # Plaintext modulus. Encrypted operations happen modulo t
                        #  Must be prime such that t-1 be divisible by 2^N.
    't_bits': 22,       # Number of bits in t. Used to generate a suitable value 
                        #  for t. Overrides t if specified.
                        #  NOTE: Standard value of 20 was not sufficient for 100x100x100 multiplication
                        #  -> increased to 22
    'sec': 128,         # Security parameter. The equivalent length of AES key in bits.
                        #  Sets the ciphertext modulus q, can be one of {128, 192, 256}
                        #  More means more security but also slower computation.
}
HE.contextGen(**bfv_params)  # Generate context for bfv scheme
HE.keyGen()             # Key Generation: generates a pair of public/secret keys
HE.rotateKeyGen()       # Rotate key generation --> Allows rotation/shifting
HE.relinKeyGen()        # Relinearization key generation

In [4]:
class ResStorage:
    execTimeTotal = 0
    deviationTotal = 0

import time
import random
import copy

num_execs = 5
num_slots = n

ccSum = ResStorage()
ccSub = ResStorage()
ccMul = ResStorage()
cSq   = ResStorage()
cNeg  = ResStorage()
cPow  = ResStorage()
cRotR = ResStorage()
cRotL = ResStorage()
cpSum = ResStorage()
cpSub = ResStorage()
cpMul = ResStorage()

def setup():
    global v1, p1, c1, v2, p2, c2
    v1 = [0] * num_slots
    v2 = [0] * num_slots
    for i in range(num_slots):
        v1[i] = random.randint(1,100)
        #v1[i] = 100
    p1 = HE.encode(v1)
    c1 = HE.encryptPtxt(p1)

    for i in range(num_slots):
        v2[i] = random.randint(1,100)
        #v2[i] = 100
    p2 = HE.encode(v2)
    c2 = HE.encryptPtxt(p2)
    
# Multiplication
def fun_ccMul():
    setup()
    start = time.time()
    
    c3 = c1 * c2
    ~c3
    c3 = c3 * c2
    ~c3
    
    end = time.time()
    ccMul.execTimeTotal += end - start

    res = HE.decrypt(c3)
    for i in range(num_slots):
        actualRes = v1[i] * v2[i] * v2[i]
        ccMul.deviationTotal += abs(res[i] - actualRes)
    
# Addition
def fun_ccSum():
    setup()
    start = time.time()
    
    c3 = c1 + c2
    c3 = c3 + c2
    
    end = time.time()
    ccSum.execTimeTotal += end - start
    
    res = HE.decrypt(c3)
    for i in range(num_slots):
        actualRes = v1[i] + v2[i] + v2[i]
        ccSum.deviationTotal += abs(res[i] - actualRes)
    
# Subtraction
def fun_ccSub():
    setup()
    start = time.time()
    
    c3 = c1 - c2
    c3 = c3 - c2
    
    end = time.time()
    ccSub.execTimeTotal += end - start
    
    res = HE.decrypt(c3)
    for i in range(num_slots):
        actualRes = v1[i] - v2[i] - v2[i]
        ccSub.deviationTotal += abs(res[i] - actualRes)
    
# Squaring
def fun_cSq():
    setup()
    start = time.time()
    
    c3 = c1 ** 2
    
    end = time.time()
    cSq.execTimeTotal += end - start
    
    res = HE.decrypt(c3)
    for i in range(num_slots):
        actualRes = pow(v1[i],2)
        cSq.deviationTotal += abs(res[i] - actualRes)
    
# Negation
def fun_cNeg():
    setup()
    start = time.time()
    
    c3 = copy.deepcopy(c1)
    -c3
    -c3
    
    end = time.time()
    cNeg.execTimeTotal += end - start
    
    res = HE.decrypt(c3)
    for i in range(num_slots):
        actualRes = v1[i]
        cNeg.deviationTotal += abs(res[i] - actualRes)
    
# Left Rotation
def fun_cRotL():
    setup()
    start = time.time()
    
    c3 = c1 << 1
    c3 = c3 << 1
    
    end = time.time()
    cRotL.execTimeTotal += end - start
    
    res = HE.decrypt(c3)
    for i in range(num_slots):
        actualRes = 0
        if (i + 2 >= num_slots):
            actualRes = v1[i + 2 - num_slots]
        else:
            actualRes = v1[i + 2]
        cRotL.deviationTotal += abs(res[i] - actualRes)
    
# Right Rotation
def fun_cRotR():
    setup()
    start = time.time()
    
    c3 = c1 >> 1
    c3 = c3 >> 1
    
    end = time.time()
    cRotR.execTimeTotal += end - start
    
    res = HE.decrypt(c3)
    for i in range(num_slots):
        actualRes = 0
        if (i - 2 < 0):
            actualRes = v1[num_slots + (i - 2)]
        else:
            actualRes = v1[i - 2]
        cRotR.deviationTotal += abs(res[i] - actualRes)
    
# Ciphertext-Plain Addition
def fun_cpSum():
    setup()
    start = time.time()
    
    c3 = c1 + p2
    c3 = c3 + p2
    
    end = time.time()
    cpSum.execTimeTotal += end - start
    
    res = HE.decrypt(c3)
    for i in range(num_slots):
        actualRes = v1[i] + v2[i] + v2[i]
        cpSum.deviationTotal += abs(res[i] - actualRes)
    
# Ciphertext-Plain Subtraction
def fun_cpSub():
    setup()
    start = time.time()
    
    c3 = c1 - p2
    c3 = c3 - p2
    
    end = time.time()
    cpSub.execTimeTotal += end - start
    
    res = HE.decrypt(c3)
    for i in range(num_slots):
        actualRes = v1[i] - v2[i] - v2[i]
        cpSub.deviationTotal += abs(res[i] - actualRes)
    
# Ciphertext-Plain Multiplication
def fun_cpMul():
    setup()
    start = time.time()
    
    c3 = c1 * p2
    c3 = c3 * p2
    
    end = time.time()
    cpMul.execTimeTotal += end - start
    
    res = HE.decrypt(c3)
    for i in range(num_slots):
        actualRes = v1[i] * v2[i] * v2[i]
        cpMul.deviationTotal += abs(res[i] - actualRes)
    
if __name__ == "__main__":
    
    for i in range(num_execs):
        fun_ccSum()
    for i in range(num_execs):
        fun_ccSub()
    for i in range(num_execs):
        fun_ccMul()
    for i in range(num_execs):
        fun_cSq()
    for i in range(num_execs):
        fun_cNeg()
    for i in range(num_execs):
        fun_cRotL()
    for i in range(num_execs):
        fun_cRotR()
    for i in range(num_execs):
        fun_cpSum()
    for i in range(num_execs):
        fun_cpSub()
    for i in range(num_execs):
        fun_cpMul()

In [5]:
    print('Execution time in milliseconds:')
    print()
    print('Addition:')    # Formula: Total time or deviation for all cycles / operations per cycle
    print('Average execution time: {:.6f}'.format( ( ccSum.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   ccSum.deviationTotal / (num_execs*2) ))
    print()
    print('Subtraction:')
    print('Average execution time: {:.6f}'.format( ( ccSub.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   ccSub.deviationTotal / (num_execs*2) ))
    print()
    print('Multiplication:')
    print('Average execution time: {:.6f}'.format( ( ccMul.execTimeTotal / (num_execs*2) ) * 1000 )) # KOMMENTIEREN
    print('Average deviation:      {:.6f}'.format(   ccMul.deviationTotal / (num_execs*2) ))
    print()
    print('Squaring:')    # Square only has 1 operation per cycle, otherwise the configuration needs to be changed
    print('Average execution time: {:.6f}'.format( ( cSq.execTimeTotal / (num_execs) ) * 1000 )) # ALLE SQUARES ÜBERPRÜFEN
    print('Average deviation:      {:.6f}'.format(   cSq.deviationTotal / (num_execs) ))
    print()
    print('Negation:')
    print('Average execution time: {:.6f}'.format( ( cNeg.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   cNeg.deviationTotal / (num_execs*2) ))
    print()
    print('Left Rotation:')
    print('Average execution time: {:.6f}'.format( ( cRotL.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   cRotL.deviationTotal / (num_execs*2) ))
    print()
    print('Right Rotation:')
    print('Average execution time: {:.6f}'.format( ( cRotR.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   cRotR.deviationTotal / (num_execs*2) ))
    print()
    print('Ciphertext-Plain Addition:')
    print('Average execution time: {:.6f}'.format( ( cpSum.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   cpSum.deviationTotal / (num_execs*2) ))
    print()
    print('Ciphertext-Plain Subtraction:')
    print('Average execution time: {:.6f}'.format( ( cpSub.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   cpSub.deviationTotal / (num_execs*2) ))
    print()
    print('Ciphertext-Plain Multiplication:')
    print('Average execution time: {:.6f}'.format( ( cpMul.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   cpMul.deviationTotal / (num_execs*2) ))
    
    PRINT_RAW_NUMBERS = True
    if (PRINT_RAW_NUMBERS):
        print()
        print()
        print('Raw numbers:')
        print()
        print('Average execution time:')
        print('{:.6f}'.format( ( ccSum.execTimeTotal / (num_execs*2) ) * 1000 ))
        print('{:.6f}'.format( ( ccSub.execTimeTotal / (num_execs*2) ) * 1000 ))
        print('{:.6f}'.format( ( ccMul.execTimeTotal / (num_execs*2) ) * 1000 ))
        print('{:.6f}'.format( ( cSq.execTimeTotal / (num_execs) ) * 1000 ))
        print('{:.6f}'.format( ( cNeg.execTimeTotal / (num_execs*2) ) * 1000 ))
        print('{:.6f}'.format( ( cRotL.execTimeTotal / (num_execs*2) ) * 1000 ))
        print('{:.6f}'.format( ( cRotR.execTimeTotal / (num_execs*2) ) * 1000 ))
        print('{:.6f}'.format( ( cpSum.execTimeTotal / (num_execs*2) ) * 1000 ))
        print('{:.6f}'.format( ( cpSub.execTimeTotal / (num_execs*2) ) * 1000 ))
        print('{:.6f}'.format( ( cpMul.execTimeTotal / (num_execs*2) ) * 1000 ))
        print()
        print('Average deviation:')
        print('{:.6f}'.format(   ccSum.deviationTotal / (num_execs*2) ))
        print('{:.6f}'.format(   ccSub.deviationTotal / (num_execs*2) ))
        print('{:.6f}'.format(   ccMul.deviationTotal / (num_execs*2) ))
        print('{:.6f}'.format(   cSq.deviationTotal / (num_execs) ))
        print('{:.6f}'.format(   cNeg.deviationTotal / (num_execs*2) ))
        print('{:.6f}'.format(   cRotL.deviationTotal / (num_execs*2) ))
        print('{:.6f}'.format(   cRotR.deviationTotal / (num_execs*2) ))
        print('{:.6f}'.format(   cpSum.deviationTotal / (num_execs*2) ))
        print('{:.6f}'.format(   cpSub.deviationTotal / (num_execs*2) ))
        print('{:.6f}'.format(   cpMul.deviationTotal / (num_execs*2) ))

Execution time in milliseconds:

Addition:
Average execution time: 0.217891
Average deviation:      0.000000

Subtraction:
Average execution time: 0.117540
Average deviation:      0.000000

Multiplication:
Average execution time: 19.905734
Average deviation:      0.000000

Squaring:
Average execution time: 10.508156
Average deviation:      0.000000

Negation:
Average execution time: 17.125487
Average deviation:      0.000000

Left Rotation:
Average execution time: 5.292392
Average deviation:      66.600000

Right Rotation:
Average execution time: 4.668379
Average deviation:      65.600000

Ciphertext-Plain Addition:
Average execution time: 0.239754
Average deviation:      0.000000

Ciphertext-Plain Subtraction:
Average execution time: 0.240755
Average deviation:      0.000000

Ciphertext-Plain Multiplication:
Average execution time: 2.369285
Average deviation:      0.000000


Raw numbers:

Average execution time:
0.217891
0.117540
19.905734
10.508156
17.125487
5.292392
4.668379
0.23975